## Assistant API - Function Calling
An assistant is a purpose-built AI that has specific instructions, leverages extra knowledge, and can call models and tools to perform tasks.

https://platform.openai.com/docs/assistants/tools/function-calling

https://cookbook.openai.com/examples/assistants_api_overview_python

https://dev.to/esponges/build-the-new-openai-assistant-with-function-calling-52f5

https://community.openai.com/t/function-calling-with-assistants-api/488259/2

https://community.openai.com/t/function-calling-with-assistants-api/488259

https://dev.to/airtai/function-calling-and-code-interpretation-with-openais-assistant-api-a-quick-and-simple-tutorial-5ce5

In [1]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv('C:/Code_Apps/Learn-Generative-AI/03_chatgpt/.env'))

api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
# Function defining for Function Calling
import json

def get_local_weather(city:str,unit:str="fahrenheit")->str:
    """Giving current local cities temperature"""
    if "lahore" in city.lower():
        return json.dumps({"city":"Lahore","temperature":"51","unit":"fahrenheit"})
    elif "karachi" in city.lower():
        return json.dumps({"city":"Karachi","temperature":"24","unit":"celsius"})
    elif "gwadar" in city.lower():
        return json.dumps({"city":"Gwadar","temperature":"14","unit":"fahrenheit"})
    else:
        return json.dumps({"city":city,"temperature":"Doesn't have given city data right now,but available in future"})


def get_nickname(city:str):
    """Get Nickname of a city"""
    if "lahore" in city.lower():
        return "LHR"
    elif "karachi" in city.lower():
        return "KHI"
    elif "gwadar" in city.lower():
        return "GWD"
    else:
        return city.lower()

### Creating Assistant and Register functions to it

In [4]:
from openai.types.beta.assistant import Assistant

assistant : Assistant = client.beta.assistants.create(
    name="weather_assistant",
    instructions="You are a weather bot. Please give information from the function provided.",
    model="gpt-3.5-turbo-1106",
    tools=[
    {
        "type":"function",
        "function":{
            "name":"getCurrentWeather",
            "description": "Get current the weather of the given city",
            "parameters":{
                "type": "object",
                "properties":{
                    "city": {"type":"string","description":"The city or location e.g Faislabad,San Francisco"},
                    "unit": {"type": "string",
                            "enum":["c","f"]
                        }
                },
                "required": ["city"],
            }
        }
    },
    {
        "type": "function",
        "function":{
            "name": "getNickname",
            "description": "Get nickname of a city",
            "parameters":{
                "type": "object",
                "properties":{
                  "city": {"type":"string","description":"The city or location e.g Faislabad,San Francisco"},
                },
                "required": ["city"]
            }
        }
    }
    ]
)

display(dict(assistant))

{'id': 'asst_0Vo7m8tUDBxiRJf2CJ622l9s',
 'created_at': 1707628844,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Please give information from the function provided.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'weather_assistant',
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='getCurrentWeather', description='Get current the weather of the given city', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city or location e.g Faislabad,San Francisco'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['city']}), type='function'),
  ToolFunction(function=FunctionDefinition(name='getNickname', description='Get nickname of a city', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city or location e.g Faislabad,San Francisco'}}, 'required': ['city']}), type='function')]}

### Create A Thread

In [5]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

display(dict(thread))

{'id': 'thread_MTrEZeJ2GDdRDB9093EZUWGp',
 'created_at': 1707628852,
 'metadata': {},
 'object': 'thread'}

### Step 3: Add Message To Thread

In [6]:
from openai.types.beta.threads.thread_message import ThreadMessage

message : ThreadMessage = client.beta.threads.messages.create(
    role="user",
    content="What is the current Weather in Gwadar",
    thread_id=thread.id
)
dict(message)

{'id': 'msg_V6tIKEb4drzQGhf69Mev8gb8',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What is the current Weather in Gwadar'), type='text')],
 'created_at': 1707628858,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_MTrEZeJ2GDdRDB9093EZUWGp'}

In [7]:
from openai.types.beta.threads.run import Run
run : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# cancel_run = client.beta.threads.runs.cancel(
#   thread_id=thread.id,
#   run_id=run.id
# )

display(dict(run))
display(run.tools)
# run_vLpWiqcN6hX4GmXJgdBOVt9U

{'id': 'run_6r9RhwG7JfHzGdzwXgygndmT',
 'assistant_id': 'asst_0Vo7m8tUDBxiRJf2CJ622l9s',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1707628862,
 'expires_at': 1707629462,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Please give information from the function provided.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_MTrEZeJ2GDdRDB9093EZUWGp',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', description='Get current the weather of the given city', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city or location e.g Faislabad,San Francisco'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['city']}), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', descriptio

[ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', description='Get current the weather of the given city', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city or location e.g Faislabad,San Francisco'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['city']}), type='function'),
 ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', description='Get nickname of a city', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The city or location e.g Faislabad,San Francisco'}}, 'required': ['city']}), type='function')]

### Run Life Cycle

![ALT TEXT](https://raw.githubusercontent.com/panaverse/learn-generative-ai/788f968387b0ad38bc379c3a4718400e8e42948d/03_chatgpt/08_assistants_function_calling/diagram.png)

### Status Definition

https://platform.openai.com/docs/assistants/how-it-works/runs-and-run-steps

##### queued:

When Runs are first created or when you complete the required_action, they are moved to a queued status. They should almost immediately move to in_progress.

##### in_progress:

While `in_progress`, the Assistant uses the model and tools to perform steps. You can view progress being made by the Run by examining the Run Steps.

##### completed:

The Run successfully completed! You can now view all Messages the Assistant added to the Thread, and all the steps the Run took. You can also continue the conversation by adding more user Messages to the Thread and creating another Run.

##### requires_action:

When using the Function calling tool, the Run will move to a required_action state once the model determines the names and arguments of the functions to be called. You must then run those functions and submit the outputs before the run proceeds. If the outputs are not provided before the expires_at timestamp passes (roughly 10 mins past creation), the run will move to an expired status.

##### expired:

This happens when the function calling outputs were not submitted before expires_at and the run expires. Additionally, if the runs take too long to execute and go beyond the time stated in expires_at, our systems will expire the run.

##### cancelling:

You can attempt to cancel an in_progress run using the Cancel Run endpoint. Once the attempt to cancel succeeds, status of the Run moves to cancelled. Cancellation is attempted but not guaranteed. cancelled Run was successfully cancelled.

##### failed:

You can view the reason for the failure by looking at the last_error object in the Run. The timestamp for the failure will be recorded under failed_at.

### Polling for updates
In order to keep the status of your run up to date, you will have to periodically retrieve the Run object. You can check the status of the run each time you retrieve the object to determine what your application should do next. We plan to add support for streaming to make this simpler in the near future.

#### Thread locks
When a Run is in_progress and not in a terminal state, the Thread is locked. This means that:

New Messages cannot be added to the Thread.
New Runs cannot be created on the Thread.


### Run Steps
![ALT TEXT](https://raw.githubusercontent.com/panaverse/learn-generative-ai/788f968387b0ad38bc379c3a4718400e8e42948d/03_chatgpt/08_assistants_function_calling/diagram-2.png)


Most of the interesting detail in the Run Step object lives in the step_details field. There can be two types of step details:

1. message_creation: This Run Step is created when the Assistant creates a Message on the Thread.
2. tool_calls: This Run Step is created when the Assistant calls a tool. Details around this are covered in the relevant sections of the Tools guide.

In [8]:
available_functions = {
    "getCurrentWeather": get_local_weather,
    "getNickname": get_nickname
}

In [11]:
import time

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

while True:

    # runStatus = checkStatus(run.id,thread.id)
    runStatus = client.beta.threads.runs.retrieve(run_id=run.id,thread_id=thread.id)
    print(runStatus.status ,'.....')

    if runStatus.status == "requires_action":
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            # print("toolCalls present:")
            toolcalls = runStatus.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

        for tools in toolcalls:
            # print('tools are: ',tools)
            # print('tool ids are: ',tools.id)
            function_name = tools.function.name
            function_args = json.loads(tools.function.arguments)
            function_to_call = available_functions[tools.function.name]
            # print(function_to_call.__name__)
            if function_name in available_functions:
                if function_to_call.__name__ == "get_local_weather":
                    # print('gettin nickweather')
                    response = function_to_call(
                        city=function_args['city'],
                        unit=function_args.get('unit')
                    )
                    # After calling function, response appended
                    tool_outputs.append({
                        "tool_call_id":tools.id,
                        "output":response
                        })
                    # print(tool_outputs)

                elif function_to_call.__name__ == "get_nickname":
                    # print('gettin nickname')
                    response = function_to_call(
                        city=function_args.get('city')
                    )
                    tool_outputs.append({
                        "tool_call_id":tools.id,
                        "output":response
                    })

                # print(tool_outputs)
        
        client.beta.threads.runs.submit_tool_outputs(
            run_id=run.id,
            thread_id=thread.id,
            tool_outputs=tool_outputs
        )

    elif runStatus.status == 'completed':
        AllMessages = client.beta.threads.messages.list(thread_id=thread.id)
        for message in reversed(AllMessages.data):
            print(f"{message.role}: {message.content[0].text.value}")
            # print(f"Message:{message.content[0].text.value} ")
        break


    elif runStatus.status == 'failed':
        print('Run status failed')
        break

    elif runStatus.status in ['queued','in_progress']:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(5)

    else:
        print("Unexpected Error")
        print(run.status)
        break
                
        
        


completed .....
user: What is the current Weather in Gwadar
assistant: The current weather in Gwadar is 14°F.
